In [1]:
"""
Interactive NHL Data Explorer with ipywidgets
Must be run in a Jupyter notebook
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

class InteractiveNHLExplorer:
    """Interactive explorer for NHL data using ipywidgets"""
    
    def __init__(self):
        # Path to the main dataset (update this if needed)
        data_path = Path(r"C:\Users\AgeTeQ\Desktop\data\classes\DS\tp1\project-template\data\tidy\all_seasons_combined.csv")
        self.data = self._load_data(data_path)
        self.current_game_data = None
        
    def _load_data(self, data_path):
        """Load the dataset from CSV"""
        if not data_path.exists():
            print(f"Data file not found: {data_path}")
            return pd.DataFrame()
        
        try:
            df = pd.read_csv(data_path)
            print(f"Loaded {len(df)} rows")
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return pd.DataFrame()
    
    def create_interactive_widget(self):
        """Create and display the interactive widgets"""
        if self.data.empty:
            print("No data available.")
            return
        
        # Dropdown for season
        seasons = sorted(self.data['season'].unique()) if 'season' in self.data.columns else ['All Seasons']
        # Dropdown for game type
        game_types = sorted(self.data['game_type'].unique()) if 'game_type' in self.data.columns else ['General']
        
        self.season_dropdown = widgets.Dropdown(
            options=seasons,
            value=seasons[0],
            description='Season:',
            style={'description_width': 'initial'}
        )
        
        self.game_type_dropdown = widgets.Dropdown(
            options=game_types,
            value=game_types[0],
            description='Game Type:',
            style={'description_width': 'initial'}
        )
        
        self.game_dropdown = widgets.Dropdown(
            description='Game ID:',
            style={'description_width': 'initial'}
        )
        
        self.event_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=100,
            step=1,
            description='Event Index:',
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.output = widgets.Output()
        
        # Handlers
        def on_season_change(change):
            self._update_game_list()
            
        def on_game_type_change(change):
            self._update_game_list()
            
        def on_game_change(change):
            self._update_event_slider()
            self._plot_current_event()
            
        def on_event_change(change):
            self._plot_current_event()
        
        # Connect handlers
        self.season_dropdown.observe(on_season_change, names='value')
        self.game_type_dropdown.observe(on_game_type_change, names='value')
        self.game_dropdown.observe(on_game_change, names='value')
        self.event_slider.observe(on_event_change, names='value')
        
        # Layout
        display(widgets.VBox([
            widgets.HBox([self.season_dropdown, self.game_type_dropdown]),
            self.game_dropdown,
            self.event_slider,
            self.output
        ]))
        
        # Initialize
        self._update_game_list()
    
    def _update_game_list(self):
        """Update the game dropdown based on season and game type"""
        season = self.season_dropdown.value
        game_type = self.game_type_dropdown.value
        
        games = self.data[
            (self.data['season'] == season) & 
            (self.data['game_type'] == game_type)
        ]['game_id'].unique()
        
        self.game_dropdown.options = sorted(games)
        if len(games) > 0:
            self.game_dropdown.value = games[0]
    
    def _update_event_slider(self):
        """Update the event slider to match the number of events in a game"""
        game_id = self.game_dropdown.value
        if game_id:
            game_data = self.data[self.data['game_id'] == game_id]
            self.current_game_data = game_data.reset_index(drop=True)
            max_events = len(game_data) - 1 if len(game_data) > 0 else 0
            self.event_slider.max = max_events
            self.event_slider.value = 0
    
    def _plot_current_event(self):
        """Plot the current selected event"""
        with self.output:
            clear_output(wait=True)
            
            if (self.current_game_data is None or 
                len(self.current_game_data) == 0 or 
                self.game_dropdown.value is None):
                print("No game data available. Please select a game.")
                return
            
            event_index = self.event_slider.value
            if event_index >= len(self.current_game_data):
                event_index = len(self.current_game_data) - 1
                self.event_slider.value = event_index
            
            event = self.current_game_data.iloc[event_index]
            
            # Make three plots
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
            
            # Plot 1: Current event
            self._draw_ice_rink(ax1)
            self._plot_single_event(ax1, event, event_index)
            
            # Plot 2: All events in the game
            self._draw_ice_rink(ax2)
            self._plot_all_events(ax2)
            
            # Plot 3: Event details
            self._display_event_info(ax3, event, event_index)
            
            plt.tight_layout()
            plt.show()
    
    def _draw_ice_rink(self, ax):
        """Draw the rink (from image if available, otherwise simple lines)"""
        rink_path = r"C:\Users\AgeTeQ\Desktop\data\classes\DS\tp1\project-template\figures\nhl_rink.png"
        
        try:
            rink_img = plt.imread(rink_path)
            ax.imshow(rink_img, extent=[-100, 100, -42.5, 42.5])
            ax.set_xlim(-100, 100)
            ax.set_ylim(-42.5, 42.5)
            ax.grid(False)
        except:
            # Simple version if no rink image
            ax.set_xlim(-100, 100)
            ax.set_ylim(-42.5, 42.5)
            rink = patches.Rectangle((-100, -42.5), 200, 85, linewidth=2, 
                                   edgecolor='black', facecolor='lightblue', alpha=0.3)
            ax.add_patch(rink)
            ax.axvline(x=-25, color='blue', linestyle='-', alpha=0.5)
            ax.axvline(x=25, color='blue', linestyle='-', alpha=0.5)
            ax.axvline(x=-89, color='red', linestyle='-', alpha=0.7)
            ax.axvline(x=89, color='red', linestyle='-', alpha=0.7)
            ax.grid(True, alpha=0.3)
        
        ax.set_aspect('equal')
        ax.set_title('Current Event Location', fontsize=12)
    
    def _plot_single_event(self, ax, event, event_index):
        """Plot a single event"""
        if pd.notna(event.get('x_coord')) and pd.notna(event.get('y_coord')):
            x, y = event['x_coord'], event['y_coord']
            color = 'red' if event.get('is_goal') == 1 else 'blue'
            marker = '*' if event.get('is_goal') == 1 else 'o'
            size = 200 if event.get('is_goal') == 1 else 120
            
            ax.scatter(x, y, c=color, marker=marker, s=size, alpha=0.9, 
                      edgecolors='white', linewidth=2, zorder=5)
            
            ax.annotate(f"Event {event_index + 1}", 
                       xy=(x, y), xytext=(x + 8, y + 8),
                       fontsize=9, fontweight='bold',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.9))
        else:
            ax.text(0, 0, "No coordinates available", ha='center', va='center',
                   fontsize=12, bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.8))
    
    def _plot_all_events(self, ax):
        """Plot all shots and goals for the game"""
        if self.current_game_data is None:
            return
            
        shots = self.current_game_data[self.current_game_data['is_goal'] == 0]
        goals = self.current_game_data[self.current_game_data['is_goal'] == 1]
        
        if not shots.empty:
            ax.scatter(shots['x_coord'], shots['y_coord'], 
                      c='blue', alpha=0.6, s=50, label=f'Shots ({len(shots)})')
        
        if not goals.empty:
            ax.scatter(goals['x_coord'], goals['y_coord'], 
                      c='red', alpha=1.0, s=100, marker='*', label=f'Goals ({len(goals)})')
        
        ax.legend()
        ax.set_title(f'All Events in Game\nShots: {len(shots)}, Goals: {len(goals)}')
    
    def _display_event_info(self, ax, event, event_index):
        """Show detailed info for the selected event"""
        ax.axis('off')
        
        if self.current_game_data is not None:
            info_text = f"""
EVENT {event_index + 1} of {len(self.current_game_data)}

GAME: {event.get('game_id', 'N/A')}
SEASON: {event.get('season', 'N/A')}
TYPE: {event.get('game_type', 'N/A')}

EVENT: {event.get('event_type', 'N/A')}
PERIOD: {event.get('period', 'N/A')}
TIME: {event.get('period_time', 'N/A')}
TEAM: {event.get('team_name', 'N/A')}
PLAYER: {event.get('shooter_name', event.get('scorer_name', 'N/A'))}
SHOT TYPE: {event.get('shot_type', 'N/A')}

COORDINATES:
X: {event.get('x_coord', 'N/A')}
Y: {event.get('y_coord', 'N/A')}
DISTANCE: {event.get('distance_from_net', 'N/A')} ft
"""
        
            ax.text(0.05, 0.95, info_text, transform=ax.transAxes, fontsize=10, 
                   verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle="round,pad=1", facecolor="lightgray", alpha=0.8))
        
        ax.set_title('Event Information', fontsize=12, pad=20)

def launch_interactive_tool():
    """Start the tool"""
    explorer = InteractiveNHLExplorer()
    explorer.create_interactive_widget()
    return explorer


In [3]:
# Cell 1: Setup
%matplotlib inline
import matplotlib.pyplot as plt

# Set default figure size and font size
plt.rcParams['figure.figsize'] = [20, 6]
plt.rcParams['font.size'] = 10

# Cell 2: Launch tool
print("Starting NHL Explorer")
explorer = launch_interactive_tool()


Starting NHL Explorer
✅ Loaded 636877 events
